In [ ]:
import import_ipynb
import pygame
from settings import *
from support import *
from timer import Timer
from monster import Monster, Opponent
from random import choice
from random import randint
from ui import UI

class Game:
    def __init__(self):
        pygame.init()
        self.display_surface = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
        pygame.display.set_caption('Monster Battle')
        self.clock = pygame.time.Clock()
        self.running = True
        # using the dictionary made from import_assets()
        self.import_assets()
        self.player_active = True

        # useless
        # groups 
        self.all_sprites = pygame.sprite.Group()

        # useless
        # data; imports from monster.ipyn; make sure names in list alligns with the file names u r importing
        player_monster_list = ['Sparchu', 'Cleaf', 'Jacana']

        # useless
        # player
        self.player_monsters = [Monster(name, self.back_surfs[name]) for name in player_monster_list]
        self.monster = self.player_monsters[randint(0,2)]
        self.all_sprites.add(self.monster)
        
        # useless
        # opponent
        opponent_name = choice(list(MONSTER_DATA.keys()))
        self.opponent_monsters = Opponent(opponent_name, self.front_surfs[opponent_name],self.all_sprites)

        # alive & broken status
        self.player_alive = True
        self.opponent1_alive = True
        self.opponent2_alive = True
        self.opponent3_alive = True
        
        self.player_broken = False
        self.opponent1_broken = False
        self.opponent2_broken = False 
        self.opponent3_broken = False

        # list of moves made against a player
        self.player_def = []
        self.opponent1_def = []
        self.opponent2_def = []
        self.opponent3_def = []        
        
        # ui
        self.ui = UI(self.monster, self.get_input)

        #timers
        self.timers = {'player end': Timer(2000, func = self.opponent1_turn),
                      'opponent1 end': Timer(2000, func = self.opponent2_turn),
                      'opponent2 end': Timer(2000, func = self.opponent3_turn),
                       'opponent3 end': Timer(2000, func = self.player_turn)}

    # when we get input in ui.ipynb, we get values in main.ipynb
    def get_input(self, state, data):
        if state == 'Attack':
            pass
        else:
            self.player_active = False
        self.timers['player end'].activate()
        print(state)
        print(data)

    # getting the effect of interaction
    def apply_move(self, player_move, opponent1_move, opponent2_move, opponent3_move):

        # addition for charges
        if player_move == 'Charge':
            self.ui.player_charges += 1
        if opponent1_move == 'Charge':
            self.ui.opponent1_charges += 1
        if opponent2_move == 'Charge':
            self.ui.opponent2_charges += 1
        if opponent3_move == 'Charge':
            self.ui.opponent3_charges += 1

        # subtraction for charges
        try:
            self.ui.player_charges -= COST_DATA[player_move]
        except:
            self.ui.player_charges -= 0
        try:
            self.ui.opponent1_charges -= COST_DATA[opponent1_move]
        except:
            self.ui.opponent1_charges -= 0
        try:
            self.ui.opponent2_charges -= COST_DATA[opponent2_move]
        except:
            self.ui.opponent2_charges -= 0
        try:
            self.ui.opponent3_charges -= COST_DATA[opponent3_move]
        except:
            self.ui.opponent3_charges -= 0
            

        # effect
        if opponent1_move == 'Repair':
            self.opponent1_broken = False
        if opponent1_move in ['Shield', 'Warshield', 'Aura']:
            if self.opponent1_broken == True:
                opponent1_move = 'Teleport'  
                print('opponent 1 broken; forced to teleport')
        try:
            player_opp1_effect = INTERACTION_DATA[player_move][opponent1_move]
        except:
            player_opp1_effect = '-'  
        try:
            opp2_opp1_effect = INTERACTION_DATA[opponent2_move][opponent1_move]
        except:
            opp2_opp1_effect = '-'  
        try:
            opp3_opp1_effect = INTERACTION_DATA[opponent3_move][opponent1_move]
        except:
            opp3_opp1_effect = '-'  

        if opponent2_move == 'Repair':
            self.opponent2_broken = False
        if opponent2_move in ['Shield', 'Warshield', 'Aura']:
            if self.opponent2_broken == True:
                opponent2_move = 'Teleport'  
                print('opponent 2 broken; forced to teleport')
        try:
            player_opp2_effect = INTERACTION_DATA[player_move][opponent2_move]
        except:
            player_opp2_effect = '-'  
        try:
            opp1_opp2_effect = INTERACTION_DATA[opponent1_move][opponent2_move]
        except:
            opp1_opp2_effect = '-'  
        try:
            opp3_opp2_effect = INTERACTION_DATA[opponent3_move][opponent2_move]
        except:
            opp3_opp2_effect = '-' 

        if opponent3_move == 'Repair':
            self.opponent3_broken = False        
        if opponent3_move in ['Shield', 'Warshield', 'Aura']:
            if self.opponent3_broken == True:
                opponent3_move = 'Teleport'  
                print('opponent 3 broken; forced to teleport')
        try:
            player_opp3_effect = INTERACTION_DATA[player_move][opponent3_move]
        except:
            player_opp3_effect = '-'  
        try:
            opp1_opp3_effect = INTERACTION_DATA[opponent1_move][opponent3_move]
        except:
            opp1_opp3_effect = '-'  
        try:
            opp2_opp3_effect = INTERACTION_DATA[opponent2_move][opponent3_move]
        except:
            opp2_opp3_effect = '-' 
        

        if player_move == 'Repair':
            self.player_broken = False
        if player_move in ['Shield', 'Warshield', 'Aura']:
            if self.player_broken == True:
                player_move = 'Teleport' 
                print('player broken; forced to teleport')
        try:
            opp1_player_effect = INTERACTION_DATA[opponent1_move][player_move]
        except:
            opp1_player_effect = '-'  
        try:
            opp2_player_effect = INTERACTION_DATA[opponent2_move][player_move]
        except:
            opp2_player_effect = '-'  
        try:
            opp3_player_effect = INTERACTION_DATA[opponent3_move][player_move]
        except:
            opp3_player_effect = '-'        

        # check for overload discharge
        if self.ui.opponent1_charges < 0:
            self.opponent1_alive = False
            opp1_player_effect = '-'  
            opp1_opp2_effect = '-' 
            opp1_opp3_effect = '-' 
        if self.ui.opponent2_charges < 0:
            self.opponent2_alive = False
            opp2_player_effect = '-'  
            opp2_opp1_effect = '-' 
            opp2_opp3_effect = '-' 
        if self.ui.opponent3_charges < 0:
            self.opponent3_alive = False
            opp3_player_effect = '-'  
            opp3_opp1_effect = '-' 
            opp3_opp2_effect = '-' 
        if self.ui.player_charges < 0:
            self.player_alive = False
            player_opp1_effect = '-'  
            player_opp2_effect = '-' 
            player_opp3_effect = '-' 

        # compiling effects to said player
        if self.ui.targets[0] in ['AOE', 'Opponent 1', 'Not Opponent 2', 'Not Opponent 3']:
            self.opponent1_def.append(player_opp1_effect)
        if self.ui.targets[2] in ['AOE', 'Opponent 1', 'Not Opponent 3', 'Not Player']:
            self.opponent1_def.append(opp2_opp1_effect)
        if self.ui.targets[3] in ['AOE', 'Opponent 1', 'Not Opponent 2', 'Not Player']:
            self.opponent1_def.append(opp3_opp1_effect)        
        
        if self.ui.targets[0] in ['AOE', 'Opponent 2', 'Not Opponent 1', 'Not Opponent 3']:
            self.opponent2_def.append(player_opp2_effect)
        if self.ui.targets[1] in ['AOE', 'Opponent 2', 'Not Opponent 3', 'Not Player']:
            self.opponent2_def.append(opp1_opp2_effect)
        if self.ui.targets[3] in ['AOE', 'Opponent 2', 'Not Opponent 1', 'Not Player']:
            self.opponent2_def.append(opp3_opp2_effect)

        if self.ui.targets[0] in ['AOE', 'Opponent 3', 'Not Opponent 1', 'Not Opponent 2']:
            self.opponent3_def.append(player_opp3_effect)
        if self.ui.targets[1] in ['AOE', 'Opponent 3', 'Not Opponent 2', 'Not Player']:            
            self.opponent3_def.append(opp1_opp3_effect)
        if self.ui.targets[2] in ['AOE', 'Opponent 3', 'Not Opponent 1', 'Not Player']:
            self.opponent3_def.append(opp2_opp3_effect)
        
        if self.ui.targets[1] in ['AOE', 'Player', 'Not Opponent 2', 'Not Opponent 3']:        
            self.player_def.append(opp1_player_effect)
        if self.ui.targets[2] in ['AOE', 'Player', 'Not Opponent 1', 'Not Opponent 3']:           
            self.player_def.append(opp2_player_effect)
        if self.ui.targets[3] in ['AOE', 'Player', 'Not Opponent 1', 'Not Opponent 2']:   
            self.player_def.append(opp3_player_effect)

        print(self.player_def, self.opponent1_def, self.opponent2_def, self.opponent3_def)
        
        # kill;break;aura proc;nothing;teleport proc
        if 'Kill' in self.opponent1_def:
            self.opponent1_alive = False
        if 'Breaks' in self.opponent1_def:
            self.opponent1_broken = True
            print('Opponent1 Broke')
        if 'Gain 1 Ch' in self.opponent1_def:
            for i in range(len(self.opponent1_def)):
                if self.opponent1_def[i] == 'Gain 1 Ch':
                    self.ui.opponent1_charges += 1
                    print('Aura Proc: Opponent1 gained 1 Ch')
        if opponent1_move == 'Teleport':
            for i in range(len(self.opponent1_def)):
                try:
                    if randint(1,8) <= (8*opponent1_effect[i]):
                        print('Opponent1 Teleport Successful')
                    else:
                        print('Opponent1 Teleport Failed')
                        self.opponent1_alive = False
                except:
                    1==1

        if 'Kill' in self.opponent2_def:
            self.opponent2_alive = False
        if 'Breaks' in self.opponent2_def:
            self.opponent2_broken = True
            print('Opponent2 Broke')
        if 'Gain 1 Ch' in self.opponent2_def:
            for i in range(len(self.opponent2_def)):
                if self.opponent2_def[i] == 'Gain 1 Ch':
                    self.ui.opponent2_charges += 1
                    print('Aura Proc: Opponent2 gained 1 Ch')
        if opponent2_move == 'Teleport':
            for i in range(len(self.opponent2_def)):
                try:
                    if randint(1,8) <= (8*opponent2_effect[i]):
                        print('Opponent2 Teleport Successful')
                    else:
                        print('Opponent2 Teleport Failed')
                        self.opponent2_alive = False
                except:
                    1==1

        if 'Kill' in self.opponent3_def:
            self.opponent3_alive = False
        if 'Breaks' in self.opponent3_def:
            self.opponent3_broken = True
            print('Opponent3 Broke')
        if 'Gain 1 Ch' in self.opponent3_def:
            for i in range(len(self.opponent3_def)):
                if self.opponent3_def[i] == 'Gain 1 Ch':
                    self.ui.opponent3_charges += 1
                    print('Aura Proc: Opponent3 gained 1 Ch')
        if opponent3_move == 'Teleport':
            for i in range(len(self.opponent3_def)):
                try:
                    if randint(1,8) <= (8*self.opponent3_def[i]):
                        print('Opponent3 Teleport Successful')
                    else:
                        print('Opponent3 Teleport Failed')
                        self.opponent3_alive = False
                except:
                    1==1

        if 'Kill' in self.player_def:
            self.player_alive = False
        if 'Breaks' in self.player_def:
            self.player_broken = True
            print('Player Broke')
        if 'Gain 1 Ch' in self.player_def:
            for i in range(len(self.player_def)):
                if self.player_def[i] == 'Gain 1 Ch':
                    self.ui.player_charges += 1
                    print('Aura Proc: Player gained 1 Ch')
        if player_move == 'Teleport':
            for i in range(len(self.player_def)):
                try:
                    if randint(1,8) <= (8*self.player_def[i]):
                        print('Player Teleport Successful')
                    else:
                        print('Player Teleport Failed')
                        self.player_alive = False
                except:
                    1==1
        
        self.kill()
        
        # charge count display
        print('Player has '+str(self.ui.player_charges)+' Charge/s')
        print('Opponent1 has '+str(self.ui.opponent1_charges)+' Charge/s')
        print('Opponent2 has '+str(self.ui.opponent2_charges)+' Charge/s')
        print('Opponent3 has '+str(self.ui.opponent3_charges)+' Charge/s')

        # winner display
        if self.player_alive == False and self.opponent1_alive == False and self.opponent2_alive == False and self.opponent3_alive == False:
            print('No one Wins')
        elif self.opponent1_alive == False and self.opponent2_alive == False and self.opponent3_alive == False:
            print('Player Wins')
        elif self.player_alive == False and self.opponent2_alive == False and self.opponent3_alive == False:
            print('Opponent 1 Wins')
        elif self.player_alive == False and self.opponent1_alive == False and self.opponent3_alive == False:
            print('Opponent 2 Wins')            
        elif self.player_alive == False and self.opponent1_alive == False and self.opponent2_alive == False:
            print('Opponent 3 Wins')

    
        self.ui.turn_moves = []
        self.player_def = []
        self.opponent1_def = []
        self.opponent2_def = []
        self.opponent3_def = []
        self.ui.targets = []
        

    # kill command
    def kill(self):
        if self.player_alive == False:
            print('player death sequence')
            self.ui.player_charges -= 100
        if self.opponent1_alive == False:
            print('opponent1 death sequence')
            self.ui.opponent1_charges -= 100
        if self.opponent2_alive == False:
            print('opponent2 death sequence')
            self.ui.opponent2_charges -= 100
        if self.opponent3_alive == False:
            print('opponent3 death sequence')
            self.ui.opponent3_charges -= 100
            
    
    # opponent123 all just do random attacking moves
    def opponent1_turn(self):
        OpponentATKCh = choice(self.ui.attacks)
        Attack = choice(ATTACK_DATA[OpponentATKCh])
        #experimental
        self.ui.turn_moves.append(Attack)
        print(Attack)
        if TARGET_DATA[Attack] == 'Single':
            self.ui.targets.append(choice(self.ui.single_target)) 
        elif TARGET_DATA[Attack] == 'Double':
            self.ui.targets.append(choice(self.ui.double_target)) 
        else:
            self.ui.targets.append('AOE')              
            
        self.timers['opponent1 end'].activate()

    def opponent2_turn(self):
        OpponentATKCh = choice(self.ui.attacks)
        Attack = choice(ATTACK_DATA[OpponentATKCh])
        #experimental
        self.ui.turn_moves.append(Attack)
        print(Attack)
        if TARGET_DATA[Attack] == 'Single':
            self.ui.targets.append(choice(self.ui.single_target)) 
        elif TARGET_DATA[Attack] == 'Double':
            self.ui.targets.append(choice(self.ui.double_target)) 
        else:
            self.ui.targets.append('AOE')  

        self.timers['opponent2 end'].activate()

    def opponent3_turn(self):
        OpponentATKCh = choice(self.ui.attacks)
        Attack = choice(ATTACK_DATA[OpponentATKCh])
        #experimental
        self.ui.turn_moves.append(Attack)
        print(Attack)
        if TARGET_DATA[Attack] == 'Single':
            self.ui.targets.append(choice(self.ui.single_target)) 
        elif TARGET_DATA[Attack] == 'Double':
            self.ui.targets.append(choice(self.ui.double_target)) 
        else:
            self.ui.targets.append('AOE')  
        
        self.timers['opponent3 end'].activate()

        print(self.ui.targets)
        print(self.ui.turn_moves)
        print(self.ui.turn_moves[0],self.ui.turn_moves[1],self.ui.turn_moves[2],self.ui.turn_moves[3])
        self.apply_move(self.ui.turn_moves[0], self.ui.turn_moves[1],self.ui.turn_moves[2],self.ui.turn_moves[3])

    
    def player_turn(self):
        self.player_active = True

    def update_timers(self):
        for timer in self.timers.values():
            timer.update()

    # useless
    # importing images from folders
    def import_assets(self):
        self.back_surfs = folder_importer('images', 'back')
        self.bg_surfs = folder_importer('images', 'other')
        self.front_surfs = folder_importer('images', 'front')

    # useless
    def draw_monster_floor(self):
        for sprite in self.all_sprites:
            floor_rect = self.bg_surfs['floor'].get_frect(center = sprite.rect.midbottom + pygame.Vector2(0, -10))
            self.display_surface.blit(self.bg_surfs['floor'], floor_rect)

    def run(self):
        while self.running:
            dt = self.clock.tick() / 1000
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.running = False
           
            # update (remove 'if self.player_active:' for more freedom)
            self.update_timers()
            self.all_sprites.update(dt)
            if self.player_active:
                self.ui.update()

            # useless
            # draw  
            self.display_surface.blit(self.bg_surfs['bg'],(0,0))
            self.draw_monster_floor()
            self.all_sprites.draw(self.display_surface)
            self.ui.draw()
            pygame.display.update()
        
        pygame.quit()
    
if __name__ == '__main__':
    game = Game()
    
    game.run() 

pygame-ce 2.5.0 (SDL 2.30.3, Python 3.11.7)
importing Jupyter notebook from settings.ipynb
importing Jupyter notebook from support.ipynb
importing Jupyter notebook from timer.ipynb
importing Jupyter notebook from monster.ipynb
importing Jupyter notebook from ui.ipynb
Defense
Teleport
Sniper
Sniper
Dranken
['AOE', 'Opponent 3', 'Opponent 2', 'Opponent 2']
['Teleport', 'Sniper', 'Sniper', 'Dranken']
Teleport Sniper Sniper Dranken
[] ['-'] ['-', '-'] ['-', 'Kill']
opponent3 death sequence
Player has 6 Charge/s
Opponent1 has 5 Charge/s
Opponent2 has 5 Charge/s
Opponent3 has -95 Charge/s
